# 07 pandas 合并、连接、去重、替换

Pandas具有全功能的，高性能内存中连接操作，与SQL等关系数据库非常相似

* pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,left_index=False, right_index=False, sort=True,suffixes=('_x', '_y'), copy=True, indicator=False)        
* pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,keys=None, levels=None, names=None, verify_integrity=False,copy=True)
* .replace()
* .duplicated() 

In [1]:
import numpy as np
import pandas as pd  
# 导入numpy、pandas模块

In [2]:
# merge合并 → 类似excel的vlookup

df1 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
df2 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
df3 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                    'key2': ['K0', 'K1', 'K0', 'K1'],
                    'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']})
df4 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                    'key2': ['K0', 'K0', 'K0', 'K0'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']})
print(pd.merge(df1, df2, on='key'))
print('------')
# left：第一个df
# right：第二个df
# on：参考键

print(pd.merge(df3, df4, on=['key1','key2']))
# 多个链接键

  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3
------
  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2


In [3]:
# 参数how → 合并方式

print(pd.merge(df3, df4,on=['key1','key2'], how = 'inner'))  
print('------')
# inner：默认，取交集

print(pd.merge(df3, df4, on=['key1','key2'], how = 'outer'))  
print('------')
# outer：取并集，数据缺失范围NaN

print(pd.merge(df3, df4, on=['key1','key2'], how = 'left'))  
print('------')
# left：按照df3为参考合并，数据缺失范围NaN

print(pd.merge(df3, df4, on=['key1','key2'], how = 'right'))  
# right：按照df4为参考合并，数据缺失范围NaN

  key1 key2   A   B   C   D
0   K0   K0  A0  B0  C0  D0
1   K1   K0  A2  B2  C1  D1
2   K1   K0  A2  B2  C2  D2
------
  key1 key2    A    B    C    D
0   K0   K0   A0   B0   C0   D0
1   K0   K1   A1   B1  NaN  NaN
2   K1   K0   A2   B2   C1   D1
3   K1   K0   A2   B2   C2   D2
4   K2   K1   A3   B3  NaN  NaN
5   K2   K0  NaN  NaN   C3   D3
------
  key1 key2   A   B    C    D
0   K0   K0  A0  B0   C0   D0
1   K0   K1  A1  B1  NaN  NaN
2   K1   K0  A2  B2   C1   D1
3   K1   K0  A2  B2   C2   D2
4   K2   K1  A3  B3  NaN  NaN
------
  key1 key2    A    B   C   D
0   K0   K0   A0   B0  C0  D0
1   K1   K0   A2   B2  C1  D1
2   K1   K0   A2   B2  C2  D2
3   K2   K0  NaN  NaN  C3  D3


In [4]:
# 参数 left_on, right_on, left_index, right_index → 当键不为一个列时，可以单独设置左键与右键

df1 = pd.DataFrame({'lkey':list('bbacaab'),
                   'data1':range(7)})
df2 = pd.DataFrame({'rkey':list('abd'),
                   'date2':range(3)})
print(pd.merge(df1, df2, left_on='lkey', right_on='rkey'))
print('------')
# df1以‘lkey’为键，df2以‘rkey’为键

df1 = pd.DataFrame({'key':list('abcdfeg'),
                   'data1':range(7)})
df2 = pd.DataFrame({'date2':range(100,105)},
                  index = list('abcde'))
print(pd.merge(df1, df2, left_on='key', right_index=True))
# df1以‘key’为键，df2以index为键
# left_index：为True时，第一个df以index为键，默认False
# right_index：为True时，第二个df以index为键，默认False

# 所以left_on, right_on, left_index, right_index可以相互组合：
# left_on + right_on, left_on + right_index, left_index + right_on, left_index + right_index

  lkey  data1 rkey  date2
0    b      0    b      1
1    b      1    b      1
2    b      6    b      1
3    a      2    a      0
4    a      4    a      0
5    a      5    a      0
------
  key  data1  date2
0   a      0    100
1   b      1    101
2   c      2    102
3   d      3    103
5   e      5    104


In [5]:
# 连接：concat

s1 = pd.Series([1,2,3])
s2 = pd.Series([2,3,4])
print(pd.concat([s1,s2]))
print('-----')
# 默认axis=0，行+行

s3 = pd.Series([1,2,3],index = ['a','c','h'])
s4 = pd.Series([2,3,4],index = ['b','e','d'])
print(pd.concat([s3,s4]).sort_index())
print(pd.concat([s3,s4], axis=1))
print('-----')
# axis=1,列+列，成为一个Dataframe

0    1
1    2
2    3
0    2
1    3
2    4
dtype: int64
-----
a    1
b    2
c    2
d    4
e    3
h    3
dtype: int64
     0    1
a  1.0  NaN
b  NaN  2.0
c  2.0  NaN
d  NaN  4.0
e  NaN  3.0
h  3.0  NaN
-----


E:\software\Anaconda\Anaconda\envs\Kaggle\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [6]:
# 去重 .duplicated

s = pd.Series([1,1,1,1,2,2,2,3,4,5,5,5,5])
print(s.duplicated())
print(s[s.duplicated() == False])
print('-----')
# 判断是否重复
# 通过布尔判断，得到不重复的值

s_re = s.drop_duplicates()
print(s_re)
print('-----')
# drop.duplicates移除重复
# inplace参数：是否替换原值，默认False

df = pd.DataFrame({'key1':['a','a',3,4,5],
                  'key2':['a','a','b','b','c']})
print(df.duplicated())
print(df['key2'].duplicated())
# Dataframe中使用duplicated

0     False
1      True
2      True
3      True
4     False
5      True
6      True
7     False
8     False
9     False
10     True
11     True
12     True
dtype: bool
0    1
4    2
7    3
8    4
9    5
dtype: int64
-----
0    1
4    2
7    3
8    4
9    5
dtype: int64
-----
0    False
1     True
2    False
3    False
4    False
dtype: bool
0    False
1     True
2    False
3     True
4    False
Name: key2, dtype: bool


In [7]:
# 替换 .replace

s = pd.Series(list('ascaazsd'))
print(s.replace('a', np.nan))
print(s.replace(['a','s'] ,np.nan))
print(s.replace({'a':'hello world!','s':123}))
# 可一次性替换一个值或多个值
# 可传入列表或字典

0    NaN
1      s
2      c
3    NaN
4    NaN
5      z
6      s
7      d
dtype: object
0    NaN
1    NaN
2      c
3    NaN
4    NaN
5      z
6    NaN
7      d
dtype: object
0    hello world!
1             123
2               c
3    hello world!
4    hello world!
5               z
6             123
7               d
dtype: object


## 作业

### 作业1：按要求创建Dataframe df1、df2，合并成df3

In [8]:
df1 = pd.DataFrame({'key':['a','b','c'],'values1':[0.911278, 0.228925, 0.407865]})
df2 = pd.DataFrame({'key':list('bcd'),'values2':[0.262151, 0.494427, 0.575295]})
df3 = pd.merge(df1, df2, on='key', how = 'outer')
df3

,key,values1,values2
0,a,0.911278,NaN
1,b,0.228925,0.262151
2,c,0.407865,0.494427
3,d,NaN,0.575295


### 作业2：按要求创建Dataframe df1、df2，并合并成df3

In [9]:
df1 = pd.DataFrame({'lkey':['a','b','c'],'values1':[0.911278, 0.228925, 0.407865]})
df2 = pd.DataFrame({'rkey':list('bcd'),'valuses2':[0.262151, 0.494427, 0.575295]})
df3 = pd.merge(df1, df2, left_on='lkey', right_on='rkey', how = 'left')
df3

,lkey,values1,rkey,valuses2
0,a,0.911278,NaN,NaN
1,b,0.228925,b,0.262151
2,c,0.407865,c,0.494427


### 作业3：按要求创建Dataframe df1、df2，并合并成df3

In [10]:
df1 = pd.DataFrame({'key':list('abc'),'values1':[0.900279,0.806439,0.528184]})
df2 = pd.DataFrame({'values2':[0.690851,0.904223,0.011268],'values3':[5,6,7]},index = list('bcd'))
df3 = pd.merge(df1,df2,left_on='key',right_index=True)
df3

,key,values1,values2,values3
1,b,0.806439,0.690851,5
2,c,0.528184,0.904223,6


### 作业4：按要求创建Dataframe df1、df2，并连接成df3

In [11]:
df1 = pd.DataFrame({'values1':[0.562833,0.484666,0.477601,0.193986],'values2':[0.588271,0.334503,0.769603,0.086822]},index=list('abcd'))
df2 = pd.DataFrame({'values1':[0.608897,0.019608,0.546241,0.559324],'values2':[0.110751,0.799938,0.440491,0.327882]},index=list('efgh'))
df3 = pd.concat([df1,df2])
df3

,values1,values2
a,0.562833,0.588271
b,0.484666,0.334503
c,0.477601,0.769603
d,0.193986,0.086822
e,0.608897,0.110751
f,0.019608,0.799938
g,0.546241,0.440491
h,0.559324,0.327882
